# 字元 (Char) 與字串 (String)

Julia 字元和字串的相關型別及階層關係如下。`Any` 是所有型別的最上層父型別。

In [4]:
function subtypetree(t, level=1, indent=4)
   level == 1 && println(t)
   for s in subtypes(t)
     println(join(fill(" ", level * indent)) * string(s))
     subtypetree(s, level+1, indent)
   end
end
subtypetree(AbstractChar)

AbstractChar
    Char


In [5]:
subtypetree(AbstractString)

AbstractString
    String
    SubString
    SubstitutionString
    Test.GenericString


字元是32位元長的原始型別

In [12]:
sizeof(Char)  # 4 bytes

4

字串沒有固定長度

In [22]:
sizeof(String)

ErrorException: Type String does not have a definite size.

In [34]:
sizeof("1/f")

3

In [20]:
sizeof("xsazf")

5

In [33]:
sizeof("\u2410")

3

## 1. 字元

### 1.1 字元的宣告

In [35]:
'x'

'x': ASCII/Unicode U+0078 (category Ll: Letter, lowercase)

In [36]:
# answer 為字元變數，使用 typeof 函式可以看到其型號為字元
answer = 'y'
typeof(answer)

Char

In [37]:
# 使用 \u 或 \U 加上 Unicode 編碼，可以顯示相對應的 Unicode 字元
'\u2460'

'①': Unicode U+2460 (category No: Number, other)

In [38]:
'\U2461'

'②': Unicode U+2461 (category No: Number, other)

In [39]:
# 同樣的，我們可以看到 2460 這個 Unicode 字元的型別為 Char
typeof('①')

Char

### 1.2 字元與數值的轉型

#### 將數值轉型為字元
字元編碼採用二/八/十/十六進位皆可

將 10 進位數值轉型為字元

In [40]:
Char(120)

'x': ASCII/Unicode U+0078 (category Ll: Letter, lowercase)

將 16 進位數值轉型為字元

In [47]:
Char(0x2461)

'②': Unicode U+2461 (category No: Number, other)

由於使用 Char 轉型並不會判斷字元編碼是否為合法值，這時候可以用 `isvalid()` 函式來判斷。合法的話，會傳回 true 值

In [53]:
println(isvalid(Char, 120))

true


如果編碼值為非法的話，則會傳回 false 值

In [54]:
println(isvalid(Char, 0x110000))

false


In [58]:
convert(Char, 0x110000)

'\U110000': Unicode U+110000 (category In: Invalid, too high)

#### 將字元轉型為數值

In [59]:
convert(Int64, 'x')

120

In [60]:
convert(UInt16, 'x')

0x0078

#### 判斷字元是否是 ascii 字元

要判斷字元是否是 ascii 字元，可以使用 `isascii()` 函式

In [61]:
println(isascii('x'))
println(isascii('\u2460')) # ① 字元

true
false


In [71]:
# ascii在0~127數值內
println(isascii('\u0000'))
println(isascii('\u0001'))
println(isascii('\u007f'))  # 16 * 7 + 15 = 127
println(isascii('\u0080'))

true
true
true
false


### 1.3 字元的運算與比較

字元跟字元之間可用減法，計算兩個字元間編碼的距離。但是加、乘、除法均不合法。

字元本身加減整數值，可以得到該字元往前或往後對應的字元。

In [82]:
'z' - 'a'

25

In [83]:
'z' + 'a'

MethodError: MethodError: no method matching +(::Char, ::Char)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:529
  +(!Matched::Integer, ::AbstractChar) at char.jl:224
  +(::T, !Matched::Integer) where T<:AbstractChar at char.jl:223

In [84]:
'z' * 'a' * 'l'  # 連接成字串，並非原本的字元型別

"zal"

In [85]:
'z' / 'a'

MethodError: MethodError: no method matching /(::Char, ::Char)

使用`^`可以重複同一字元並連接成字串

In [89]:
'=' ^ 10

"=========="

字元本身加減整數值，可以得到該字元往前或往後對應的字元。

In [93]:
'A' + 25

'Z': ASCII/Unicode U+005A (category Lu: Letter, uppercase)

In [94]:
'x' + 100

'Ü': Unicode U+00DC (category Lu: Letter, uppercase)

In [97]:
'c' - 2

'a': ASCII/Unicode U+0061 (category Ll: Letter, lowercase)

字元之間可以使用比較運算子進行比較

In [98]:
'A' < 'a'

true

## 2. 字串

### 2.1 字串的宣告

字串是以成對雙引號或是成對的 3 個雙引號。

In [99]:
x = "Hello Julia"
typeof(x)

String

In [100]:
# 成對的 3 個雙引號
typeof("""Hello Julia""")

String

In [101]:
# 字串中也可以包含 Unicode 字元
y = "\u2200 x \u2203 y"

"∀ x ∃ y"

若是字串中要包含引號的話, 可以用下列兩種方式:
- 成對引號中, 使用\\加引號
- 成對3引號中, 使用引號

In [102]:
println("方式1: 成對引號中, 使用\\加引號")
println("Hello \"Julia\" world\n")
println("方式2: 成對3引號中, 使用引號")
println("""Hello "Julia" world""")

方式1: 成對引號中, 使用\加引號
Hello "Julia" world

方式2: 成對3引號中, 使用引號
Hello "Julia" world


Julia 的字串也支援 C 語言的跳脫序列 (escape sequence)。下面範例是在字串中加入換行 `\n`。

詳細說明可以參考 Wikipedia [C syntax](https://en.wikipedia.org/wiki/C_syntax#Backslash_escapes)

In [103]:
str = "hello\nword"

"hello\nword"

In [104]:
println(str)

hello
word


### 2.2 字串的索引

字串可以透過索引值，取得對應位置的字元或子字串

字串的索引起始值是從 1 開始，在 Julia 語言中預設皆是如此，與其他大多程式語言有所不同

In [105]:
x = "Hello Julia"
# 取得 x 字串的第 3 個字元
x[3]

'l': ASCII/Unicode U+006C (category Ll: Letter, lowercase)

In [106]:
# 取得 x 字串的第 7 - 最後一個字元的子字串
x[7:end]

"Julia"

In [108]:
# 取得 x 字串的第 1 - 第5字元的子字串
x[1:5]  # or x[begin:5]

"Hello"

In [113]:
x[5:5] != x[5]  # 前面是新的字串，而後面是字元

true

Range indexing makes a copy of the selected part of the original string. Alternatively, it is possible to create a **view** into a string using the type `SubString`, for example:

索引值區間的語法會複製所選範圍的字串，也就是產生了新的字串。

相對的，可以使用`SubString`來產生**view**而非複製。

In [114]:
str = "long string"
substr = SubString(str, 1, 4)

"long"

In [115]:
typeof(substr)

SubString{String}

Several standard functions like `chop`, `chomp` or `strip` return a `SubString`.

#### 字串中有 UTF-8 編碼字元時計算索引位置

下列字串包含 UTF-8 字元。

In [116]:
str = "\u2200x\u2203y" 

"∀x∃y"

In [117]:
str[1]

'∀': Unicode U+2200 (category Sm: Symbol, math)

因為 UTF-8 字元長度不一定相同，所以用索引有可能無法定位到正確的字元，而會產生錯誤。

In [118]:
str[2]

StringIndexError: StringIndexError("∀x∃y", 2)

use the `eachindex` function to iterate over the valid character indices:

In [134]:
collect(eachindex(str))

4-element Array{Int64,1}:
 1
 4
 5
 8

In [129]:
prevind(str, 8, 2)  # 索引值8往前2個索引值

4

In [126]:
str[4]

'x': ASCII/Unicode U+0078 (category Ll: Letter, lowercase)

In [122]:
str[prevind(str, end, 2)]

'x': ASCII/Unicode U+0078 (category Ll: Letter, lowercase)

呼叫 `length()` 函式算出字串的長度，我們可以看到是 4 個字元組成的字串。

但是若以索引數來看的話，索引長度是 8。`firstindex()` 回傳是字串中第一個索引值，`lastindex()` 回傳是字串中最後一個索引值。

In [119]:
println("字元數目：", length(str))
println("索引長度：", lastindex(str) - firstindex(str) + 1)

字元數目：4
索引長度：8


呼叫 `nextind()` 函式，可以找出正確的 index。在下面例子中，先用 `nextind()` 找出所有在字串中每個字元正確的 index，之後再透過 index 印出每個字元。

取得的索引值為存在陣列 (array) 中，陣列我們會在接下來的內容中詳細介紹。

In [120]:
a = []
i = 0

while i < lastindex(str)
    i = nextind(str, i)
    push!(a, i)
end

In [121]:
# 列出原先字串中的每個字元
for j in a
    print(str[j])
end

∀x∃y

use the string as an iterable object

將字串視為可迭代的物件

In [123]:
for i in str
    print(i)
end

∀x∃y

#### 字串的插值 (Interpolation)

在應用上，常會遇到需要在字串中插入變數值，與字串結合或輸出，這時候我們可以在字串使用 $，$ 後接續變數名稱或是表達式 (expression)，就可以達到將字串中的變數值或是表達式整合在一起。

In [135]:
greet = "Welcome"
whom = "Julia"

print("$greet $whom")

Welcome Julia

In [136]:
"1 + 2 = $(1 + 2)"

"1 + 2 = 3"

#### 字串常用操作 – 組合

字串與字串或字元之間的組合，可以透過下列幾種方式進行操作：
- `string()` 函式
- `*` 運算子
- `^` 運算子
- Broadcast 函式

In [0]:
string("abc", "123")

"abc123"

In [0]:
"abc" * "123"

"abc123"

In [92]:
"d=" * "1a" ^ 10  # ^運算子會重複同一字串且優先序高於*運算子

"d=1a1a1a1a1a1a1a1a1a1a"

In [0]:
broadcast(*, "abc", "123")

"abc123"

In [140]:
(*).("abc", "123")

"abc123"

若是要結合字串/字元元組 (tuple) 或是陣列 (array)，可以使用 `join()` 函式，也可以使用 String() 建構子。

In [148]:
join(["abc" "123"])

"abc123"

In [149]:
String(['a', 'b', 'c'])

"abc"

#### 字串常用操作 – 比較

字串與字串之間可以用比較運算子進行比較

In [150]:
"abc" < "xyz"

true

但是字串和字元之間不能互相比較大小，只能比較是否相等

In [151]:
# 產生 exception
"abc" < 'a'

MethodError: MethodError: no method matching isless(::String, ::Char)
Closest candidates are:
  isless(!Matched::Char, ::Char) at char.jl:209
  isless(!Matched::Missing, ::Any) at missing.jl:87
  isless(!Matched::AbstractChar, ::AbstractChar) at char.jl:216
  ...

In [152]:
"abc" == 'a'

false

#### 字串常用操作 – 搜尋

要搜尋子字串是否在字串內，可使用 `occursin()` 函式

In [153]:
occursin("world", "hello world")

true

也可以搜尋字元是否在字串內

In [160]:
occursin('w', "hello world")

true

要搜尋字元是否在字串內，可使用 in 或是 ∈

In [154]:
'w' ∈ "hello world"

true

`findfirst()`, `findlast()`, `findprev()`, `findnext()` 函式

In [155]:
# 找第一個
findfirst("o", "hello world")

5:5

In [156]:
# 找最後一個
findlast("o", "hello world")

8:8

若要比較字元的話，不能直接比較，需要用下列語法進行比較。

回傳值與字串互比略為不同，回傳的是絕對的位置。

In [161]:
findlast('o', "hello world")

8

In [159]:
isequal('o')

(::Base.Fix2{typeof(isequal),Char}) (generic function with 1 method)

In [157]:
findlast(isequal('o'), "hello world")

8

In [158]:
# 找指定位置的找下一個符合字元
findnext(isequal('l'), "hello world", 6)

10

In [162]:
# 找指定位置的前一個符合字元
findprev(isequal('o'), "hello world", 5)

5

#### 字串常用操作 – 取代

取代字串中的內容，可以使用 `replace()` 函式，由於字串本身是 immutable，所以使用 `replace()` 函式不會變更原字串。

In [163]:
replace("hello world", "o"=>"p")

"hellp wprld"

搭配使用 count，可以指定要取代的數目。下例僅會取代第一個 o。

In [164]:
replace("hello world", "o"=>"p"; count=1)

"hellp world"

#### 字串常用操作 – 分割

分割字串，例如要分割一個逗號分隔的字串時，可以使用 `split()` 函式，分割後的字串，型別為 SubString。

In [165]:
split("hello, world, John, Wick", ',')

4-element Array{SubString{String},1}:
 "hello"
 " world"
 " John"
 " Wick"

#### 字串常用操作 – 與數值之間的轉型

要將數值轉型為字串，可以使用 `string()` 函式

In [166]:
string(10)

"10"

轉型時，加上 pad 參數可以補足位數

下面例子是110 之前補 7 個 0，產生總共 10 位數長的字串。

In [167]:
string(110, base=10, pad=10)

"0000000110"

要將字串轉型為數值，可使用 `parse()` 函式

In [168]:
parse(Float64, "12.3")

12.3

In [169]:
typeof(ans)

Float64

## [Regular Expressions](https://docs.julialang.org/en/v1/manual/strings/#Regular-Expressions-1)
The most basic regular expression literal without any options turned on just uses `r"..."`:

前面接r的字串即是最基本的正規表達式

由於較為複雜，所以請參考連結網頁。

In [170]:
r"^\s*(?:#|$)"

r"^\s*(?:#|$)"

In [171]:
typeof(ans)

Regex

## Raw String Literals
Raw strings without interpolation or unescaping can be expressed with non-standard string literals of the form `raw"..."`. 
字串前面接raw

`raw"..."`

In [175]:
println("\\")

\


In [184]:
println(raw" \ ")

 \ 


The exception is that quotation marks still must be escaped, e.g. `raw"\""` is equivalent to `"\""`.

In [177]:
 println(raw"\\ \\\"")

\\ \"


# References:
- Marathon example notebook
- [Strings](https://docs.julialang.org/en/v1/manual/strings/)
- [Wikipedia: UTF-8](https://en.wikipedia.org/wiki/UTF-8)
- [C syntax: Backslash escapes](https://en.wikipedia.org/wiki/C_syntax#Backslash_escapes)
- [Unicode 13.0 Character Code Charts](http://www.unicode.org/charts/)